In [ ]:
import os, zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models


local_zip = r".\cats_and_dogs_filtered.zip"
extract_to = r"."


# 압축 해제(이미 해제했다면 생략)
# with zipfile.ZipFile(local_zip, 'r') as zf:
#     zf.extractall(extract_to)

base_dir = os.path.join(extract_to, 'cats_and_dogs_filtered')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_gen = ImageDataGenerator(rescale=1/255)
val_gen = ImageDataGenerator(rescale=1/255)

train_generator = train_gen.flow_from_directory(
    train_dir, target_size=(150, 150), batch_size=20, class_mode='binary')

val_generator = val_gen.flow_from_directory(
    validation_dir, target_size=(150, 150), batch_size=20, class_mode='binary')

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=5,
    validation_data=val_generator,
    validation_steps=50
)

model.save(r"models\animals.h5")
print("animals.h5 로 저장되었습니다.")

In [19]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.models import load_model

model_path = r"models\animals.h5"
if not os.path.exists(model_path):
    print("모델 파일이 없습니다.")
    exit()

model = load_model(model_path)
print("모델 로드 완료")

def predict_image(image_path):
    if not os.path.exists(image_path):
        print(f"[오류] 파일이 없습니다: {image_path}")
        return

    img = load_img(image_path, target_size=(150, 150))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)[0][0]

    if prediction > 0.5:
        print(f"{image_path} → 강아지일 확률: {prediction:.2f}")
    else:
        print(f"{image_path} → 고양이일 확률: {1 - prediction:.2f}")


predict_image("dog.jpg" )


모델 로드 완료
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
dog.jpg → 강아지일 확률: 0.99
